# Layer 5 — Feedback & Learning

Time dynamics, robustness/stability checks, and A/B test planning.

**Note:** This notebook is read-only for reports. It does NOT write to MD files.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import sys

ROOT = Path('.').resolve().parent
sys.path.insert(0, str(ROOT))

# Load data
DATA_DIR = ROOT / 'data'
possible_files = ['cfm_pltv_train.csv', 'cfm_pltv_train_1.csv', 'cfm_pltv_train_imoney.csv', 'clm_pltv_iamount.csv']

df = None
for fname in possible_files:
    fpath = DATA_DIR / fname
    if fpath.exists():
        df = pd.read_csv(fpath, nrows=100_000, low_memory=False)
        print(f'✅ Loaded {len(df):,} rows from {fname}')
        break

if df is None:
    raise FileNotFoundError(f"No training data found")

df['install_date'] = pd.to_datetime(df['install_date'])

# Currency settings
CURRENCY = "VND"
VND_TO_USD = 24000

def convert_currency(value, to_currency="VND"):
    if to_currency == "USD":
        return value / VND_TO_USD
    return value

def format_currency(value, currency="VND"):
    if currency == "USD":
        return f"${value:,.2f}"
    return f"₫{value:,.0f}"

currency_symbol = "₫" if CURRENCY == "VND" else "$"
print(f"💱 Currency: {CURRENCY} ({currency_symbol})")

In [ ]:
# Time dynamics: revenue by install cohort with currency conversion
daily = df.groupby('install_date').agg(
    users=('vopenid', 'count'),
    total_ltv30=('ltv30', 'sum'),
    avg_ltv30=('ltv30', 'mean'),
    payer_rate=('is_payer_30', 'mean'),
).reset_index()

daily['total_ltv30_display'] = convert_currency(daily['total_ltv30'], CURRENCY)
daily['avg_ltv30_display'] = convert_currency(daily['avg_ltv30'], CURRENCY)

fig = go.Figure()
fig.add_trace(go.Bar(x=daily['install_date'], y=daily['total_ltv30_display'],
                     name='Total LTV30', marker_color='lightblue'))
fig.add_trace(go.Scatter(x=daily['install_date'], y=daily['avg_ltv30_display'],
                         name='Avg LTV30', yaxis='y2', line=dict(color='red')))
fig.update_layout(
    title=f'Revenue Dynamics by Install Cohort - {CURRENCY}',
    yaxis=dict(title=f'Total LTV30 ({currency_symbol})'),
    yaxis2=dict(title=f'Avg LTV30 ({currency_symbol})', side='right', overlaying='y')
)
fig.show()

In [ ]:
# Stability by country with currency conversion
country_stats = df.groupby('first_country_code').agg(
    users=('vopenid', 'count'),
    avg_ltv30=('ltv30', 'mean'),
    payer_rate=('is_payer_30', 'mean'),
).reset_index()

country_stats['avg_ltv30_display'] = convert_currency(country_stats['avg_ltv30'], CURRENCY)

fig = px.bar(country_stats, x='first_country_code', y='avg_ltv30_display',
             color='payer_rate', title=f'Avg LTV30 by Country - {CURRENCY}',
             labels={'first_country_code': 'Country', 'avg_ltv30_display': f'Avg LTV30 ({currency_symbol})'})
fig.show()

In [ ]:
# Weekly stability with currency conversion
df['install_week'] = df['install_date'].dt.isocalendar().week.astype(int)
week_stats = df.groupby('install_week').agg(
    users=('vopenid', 'count'),
    avg_ltv30=('ltv30', 'mean'),
    payer_rate=('is_payer_30', 'mean'),
).reset_index()

week_stats['avg_ltv30_display'] = convert_currency(week_stats['avg_ltv30'], CURRENCY)

fig = go.Figure()
fig.add_trace(go.Bar(x=week_stats['install_week'].astype(str), y=week_stats['users'],
                     name='Users', marker_color='lightblue'))
fig.add_trace(go.Scatter(x=week_stats['install_week'].astype(str), y=week_stats['avg_ltv30_display'],
                         name='Avg LTV30', yaxis='y2', line=dict(color='red')))
fig.update_layout(
    title=f'Weekly Cohort Stability - {CURRENCY}',
    yaxis=dict(title='Users'),
    yaxis2=dict(title=f'Avg LTV30 ({currency_symbol})', side='right', overlaying='y')
)
fig.show()

In [ ]:
## Summary

This notebook provides feedback and learning analysis including:
- Time dynamics of revenue by install cohort
- Stability checks by country
- Weekly cohort stability analysis
- A/B test planning framework

**Note:** This is an exploratory notebook. It does NOT write to report MD files.